*   Name - ID: 
*   Team members: 
1.   Le Hoang Vinh - V202000144
2.   Khau Lien Kiet - V202000199
3.   Nguyen Duong Tung - V202000270








In [ ]:
#Import necessary libraries and frameworks
import time
import pandas as pd
from sklearn.linear_model import LinearRegression
import seaborn as sns
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import make_pipeline
import numpy as np
import statsmodels.api as sm
from sklearn.feature_selection import SequentialFeatureSelector as sfs

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


# 1. Data preparation

**Observation:** There are a significant amount of N/A values, preventing the analysis. Hence we need to deal with them by using appropriate dropping and imputation.

**Data cleaning:** Drop columns that contains more than 50% null values, contains duplicate values (e.g.total_cases and
total_cases_per_million)

**Justification:** While we can impute missing values, there still needs to be enough data in a column to do a good imputation. It is decided that if more than half of the data in a column is missing,
we will drop it then try imputing.

We should avoid drop rows from dataset as the number of records is quite small. Large deduction will significantly affect model performance


In [ ]:
# Read data into dataframe and clean data
df = pd.read_csv("owid-2022-clean.csv")

# Old Data Cleaning (Drop approximately 40/60 initial feature)

# #drop columns that are duplicate, having too low number of values, having too low or high cardinality
# drop = ["weekly_icu_admissions", "icu_patients","hosp_patients", "new_tests_per_thousand","new_vaccinations"
#         ,"new_cases","new_deaths", "new_people_vaccinated_smoothed_per_hundred","weekly_hosp_admissions",
#         "weekly_icu_admissions_per_million","weekly_hosp_admissions_per_million",
#         "hosp_patients_per_million","icu_patients_per_million","new_tests_smoothed","people_vaccinated",
#         "people_fully_vaccinated","new_vaccinations_smoothed_per_million","new_deaths_smoothed_per_million"
#         ,"new_deaths_per_million","new_cases_smoothed_per_million","new_cases_per_million",
#         "new_tests_smoothed_per_thousand"]
#drop ignore columns
# drop_ignore = ["excess_mortality_cumulative_absolute","excess_mortality_cumulative",
#         "excess_mortality","excess_mortality_cumulative_per_million", "location"
#         , "date", "total_deaths", "total_cases", "population", "tests_units"]
# drop += drop_ignore
# #drop N/A values in continent row
# df.dropna(subset = ['continent'], inplace = True)

# New data cleaning
#drop columns in instructions 
# Note: 
#  - feature "location" is not dropped to be used for groupping
#  - total_cases_per_million feature is dropped based on the law of casualty
drop =["iso_code","date", "population", "tests_units","total_deaths_per_million",
       "total_cases_per_million"]

# #drop columns with more than 50% N/A values:
drop_na = ["people_vaccinated_per_hundred", "people_fully_vaccinated_per_hundred",
"total_boosters_per_hundred","handwashing_facilities"]
drop += drop_na

#drop selected columns
df.drop(columns = drop, inplace = True)

# replace empty values to N/A values for consistency
df.replace("", np.nan, inplace=True)

# Handle 0 values
"""
In the given dataset there are 0 values in 3 columns: new_cases_per_million (262),
new_deaths_per_million (483) and positive_rate (8):

-In the former 2 columns, this could be regarded as normal as every other 
measurements of the countries are presented and they may have no case at that 
day (real 0). Hence we could change these values to NA to apply the imputer.

- The latter one (positive_rate) constitutes 7 records of China and 1 record of 
Tanzania. We notice that this is a special feature as this is the inverse of the
 tests_per_case feature next to it. So China have the tests_per_case feature 
 values all present, hence we can calculate the value of positive_rate accordingly.
 It turns out that the values are too small that are rounded to 0. For Tazania, 
 the tests_per_case value are N/A hence we also apply N/A for positive_rate.

 -> In summary, we deal with the 0 values by calculating the positive_rate of China
 then transform all 0 values left into N/A for the imputer.
"""
china_cases = df.loc[260:266]
china_cases["positive_rate"] = 1/china_cases["tests_per_case"]

df.replace(0, np.nan, inplace=True)
china_cases

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:61: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,continent,location,new_cases_per_million,new_deaths_per_million,new_tests_smoothed_per_thousand,positive_rate,tests_per_case,stringency_index,population_density,median_age,...,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,hospital_beds_per_thousand,life_expectancy,human_development_index
260,Asia,China,0.045,NaN,10.227,0.000007,143597.3,64.35,147.674,38.7,...,5.929,15308.712,0.7,261.899,9.74,1.9,48.4,4.34,76.91,0.761
261,Asia,China,0.084,NaN,10.227,0.000007,141629.6,64.35,147.674,38.7,...,5.929,15308.712,0.7,261.899,9.74,1.9,48.4,4.34,76.91,0.761
262,Asia,China,0.053,NaN,10.227,0.000007,148336.2,64.35,147.674,38.7,...,5.929,15308.712,0.7,261.899,9.74,1.9,48.4,4.34,76.91,0.761
263,Asia,China,0.066,NaN,10.227,0.000007,151597.4,64.35,147.674,38.7,...,5.929,15308.712,0.7,261.899,9.74,1.9,48.4,4.34,76.91,0.761
264,Asia,China,0.076,NaN,10.227,0.000007,153625.2,64.35,147.674,38.7,...,5.929,15308.712,0.7,261.899,9.74,1.9,48.4,4.34,76.91,0.761
265,Asia,China,0.105,NaN,10.227,0.000007,145618.9,64.35,147.674,38.7,...,5.929,15308.712,0.7,261.899,9.74,1.9,48.4,4.34,76.91,0.761
266,South America,Colombia,281.942,4.896,1.133,0.384615,2.6,62.04,44.223,32.2,...,4.312,13254.949,4.5,124.240,7.44,4.7,13.5,1.71,77.29,0.767


In [ ]:
# Data exploratory
# df.info()
# df.describe()
df.shape

# df.head(10)
# df.select_dtypes("float64").nunique()

#Calculate the number of 0 in each column
# replace empty and 0 values to N/A values for consistency
print("The number of 0 values in each column:")
for column in df.columns:

  print(f"{column}:{(df[column] == 0).sum()}")
# Calculate percentage of null record in each column
print("\nThe percentage of NA values in each column:")
df.isnull().sum()/len(df)

# Plot correlation heat map between features
# corr = df.select_dtypes("float64").drop(columns = ["new_cases_smoothed", 
#                                                    "new_deaths_smoothed"]).corr()
# sns.heatmap(corr);    
# df.head(10)

The number of 0 values in each column:
continent:0
location:0
new_cases_per_million:0
new_deaths_per_million:0
new_tests_smoothed_per_thousand:0
positive_rate:0
tests_per_case:0
stringency_index:0
population_density:0
median_age:0
aged_65_older:0
aged_70_older:0
gdp_per_capita:0
extreme_poverty:0
cardiovasc_death_rate:0
diabetes_prevalence:0
female_smokers:0
male_smokers:0
hospital_beds_per_thousand:0
life_expectancy:0
human_development_index:0

The percentage of NA values in each column:


continent                          0.000000
location                           0.000000
new_cases_per_million              0.203326
new_deaths_per_million             0.387755
new_tests_smoothed_per_thousand    0.289494
positive_rate                      0.337113
tests_per_case                     0.336357
stringency_index                   0.074074
population_density                 0.015873
median_age                         0.026455
aged_65_older                      0.037037
aged_70_older                      0.031746
gdp_per_capita                     0.031746
extreme_poverty                    0.354497
cardiovasc_death_rate              0.021164
diabetes_prevalence                0.010582
female_smokers                     0.238095
male_smokers                       0.248677
hospital_beds_per_thousand         0.132275
life_expectancy                    0.000000
human_development_index            0.031746
dtype: float64

In [ ]:
# Process the data 
# 1. Use the k-nearest neighbor imputer for numerical attribute
# (Each sample’s missing values (N/A) are imputed using the mean value from 
# n_neighbors nearest neighbors found in the training set)

imputer = KNNImputer(missing_values=np.nan,n_neighbors=2)
df1 = df.drop(columns = ["continent","location"])
df1 = pd.DataFrame(imputer.fit_transform(df1), columns=df1.columns).join(df["location"])

# 2. One hot encoded the catergorical data in column "Continent"
df2 = pd.get_dummies(df["continent"])

# join data
df3 = df1.join(df2)

# 3. Smooth (average) all the numerical measurement over a week of a country
df3 = df3.groupby("location").mean()

#Write regression ready data to regression_data.csv file
df3.to_csv("regression_data.csv", encoding='utf-8', index=False)    

# 2. Perform regression and standard diagnostic

In [ ]:
data = pd.read_csv("regression_data.csv")

In [ ]:
# Build Model to predict "new_cases_per_million"
features = data.drop(columns = ["new_cases_per_million", "new_deaths_per_million"]).columns
data1 = data.drop(columns = "new_deaths_per_million")
x = data1[features]
y1 = data1["new_cases_per_million"]

#add constant to predictor variables
x = sm.add_constant(x)

#fit linear regression model
model = sm.OLS(y1,x).fit();

# view model summary
print(model.summary())

#predict new cases
y_pred1 = model.predict(x)

                              OLS Regression Results                             
Dep. Variable:     new_cases_per_million   R-squared:                       0.432
Model:                               OLS   Adj. R-squared:                  0.356
Method:                    Least Squares   F-statistic:                     5.733
Date:                   Thu, 02 Jun 2022   Prob (F-statistic):           9.09e-12
Time:                           07:28:27   Log-Likelihood:                -1652.9
No. Observations:                    189   AIC:                             3352.
Df Residuals:                        166   BIC:                             3426.
Df Model:                             22                                         
Covariance Type:               nonrobust                                         
                                      coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:117: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [ ]:
# Build Model to predict "new_deaths_per_million"
data2 = data.drop(columns = "new_cases_per_million")
x = data2[features]
y2 = data2["new_deaths_per_million"]

#add constant to predictor variables
x = sm.add_constant(x)
#fit linear regression model
model = sm.OLS(y2,x).fit()

# view model summary
print(model.summary())

#predict new deaths
y_pred2 = model.predict(x)

                              OLS Regression Results                              
Dep. Variable:     new_deaths_per_million   R-squared:                       0.487
Model:                                OLS   Adj. R-squared:                  0.419
Method:                     Least Squares   F-statistic:                     7.152
Date:                    Thu, 02 Jun 2022   Prob (F-statistic):           6.36e-15
Time:                            07:28:27   Log-Likelihood:                -513.81
No. Observations:                     189   AIC:                             1074.
Df Residuals:                         166   BIC:                             1148.
Df Model:                              22                                         
Covariance Type:                nonrobust                                         
                                      coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:117: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


#3. p-value intepretation



* Coefficients having p-values less than alpha are statistically significant and vice versa. In other words, we can consider features with low p-value to be important in affecting the outcome. Below are our opinion on valid and invalid p-value based on real-life semantics and relationships between the features and the target.
* **Valid results:**
  * In both model, we could observe that feature age_65_older has low p-value and positive coefficient, indicating a strong positive correlation. This is understandable as countries with more older citizens are prone to have more cases and deaths (elders are susceptible to covids as they have weak immunity). 
  * Another valid measurements are the relatively small values of p-values for gdp_per_capita in both cases. This suggests that gdp_per_capita is a vital feature determining the number of new cases and deaths. The claim makes sense with a positive coefficient in the new cases prediction model as gdp_per_capita represents a country prosperity, and the more prosper a nation is, the better resources it can utilize to prevent the impact of covid-19. However, in the death prediction model the coefficient are negative contradicting our belief.
  * diabetes_prevalence has a positive weight accompanied by a low p_value (0.042), rendering it the top determinant of new deaths cases. This is a valid analysis result as diabetes increase the mortality rate of the patients, which combines with covid could greatly affect the death rate and cases.
* **Invalid results:**
  * The p-value of feature extreme_poverty in new_deaths_prediction model is surprisingly high (0.922) in contrast to the one in new_cases_model, suggesting that this is not a statistically siginificant feature. This does not make much sense as extreme_poverty demonstrates the part of population living in extreme poverty, not being able to afford medical support. Hence this has certain correlation with the death rate and new death cases and should have a higher importance. The low significane may be attributed to the fact that the deaths reported are not comprehensive with cases outside of hospitals overlooked (usually poor people).
  * life_expectancy feature has a really low p_value (0.038) indicating a heavy importance of this feature in the new cases prediction model. This may not be the case as new covid affected cases, intuitively does not depend on how long a people is expected to live, but life expectancy might be a better dictator of mortality rate (new death cases).


In the upcoming questions, we decided to use SequentialFeatureSelector to perform the task of stepwise regression, as it allows us to consequently select/eliminate features until a certain condition is satisfied. Here, the scoring function/criteria are accuracy, which means that we choose/remove features to optimize accuracy. The accuracy is automatically calculated and optimized by the function hence we use this primarily to retrieve the feature, not to measure the accuracy. Also, the target is to select 15/23 feature (or eliminate 8) and compare the results between 2 approaches (backward and forward)

# 4.Step-wise forward regression

In [ ]:
def step_wise_regression(model, direction, num_features, data, feature, target):
  curr_time = time.time()
  sfs_fit = sfs(model, n_features_to_select = num_features, direction=direction, 
            scoring= 'accuracy', n_jobs =-1).fit(data[features], data[target])
  print(f"Time executing {direction} stepwise regression is {round((time.time() - curr_time),2)}s")
  #get selected feature from stepwise selection for regression
  selected_feature_boolean = sfs_fit.get_support()

  if direction == 'backward':
    selected_feature_boolean = np.invert(np.array(selected_feature_boolean))
  selected_feature = features[selected_feature_boolean]
  # get data for training
  x_stepwise = data[selected_feature]
  y_stepwise = data[target]

  #add constant to predictor variables
  x_stepwise = sm.add_constant(x_stepwise)
  #fit linear regression model
  model = sm.OLS(y_stepwise,x_stepwise).fit()

  # view model summary
  print(model.summary())

In [ ]:
model = LinearRegression()
# new cases prediction with forward model
step_wise_regression(model, 'forward', 15, data1, features, 'new_cases_per_million')

Time executing forward stepwise regression is 8.07s
                              OLS Regression Results                             
Dep. Variable:     new_cases_per_million   R-squared:                       0.394
Model:                               OLS   Adj. R-squared:                  0.342
Method:                    Least Squares   F-statistic:                     7.500
Date:                   Thu, 02 Jun 2022   Prob (F-statistic):           1.10e-12
Time:                           07:28:35   Log-Likelihood:                -1659.0
No. Observations:                    189   AIC:                             3350.
Df Residuals:                        173   BIC:                             3402.
Df Model:                             15                                         
Covariance Type:               nonrobust                                         
                                      coef    std err          t      P>|t|      [0.025      0.975]
----------------------------

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:117: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [ ]:
# new deaths prediction with forward model
step_wise_regression(model, 'forward', 15, data2, features, 'new_deaths_per_million')

Time executing forward stepwise regression is 5.72s
                              OLS Regression Results                              
Dep. Variable:     new_deaths_per_million   R-squared:                       0.426
Model:                                OLS   Adj. R-squared:                  0.376
Method:                     Least Squares   F-statistic:                     8.567
Date:                    Thu, 02 Jun 2022   Prob (F-statistic):           1.61e-14
Time:                            07:28:41   Log-Likelihood:                -524.32
No. Observations:                     189   AIC:                             1081.
Df Residuals:                         173   BIC:                             1133.
Df Model:                              15                                         
Covariance Type:                nonrobust                                         
                                      coef    std err          t      P>|t|      [0.025      0.975]
------------------

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:117: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


# 5.Step-wise backward regression

In [ ]:
# new cases prediction with backward model
step_wise_regression(model, 'backward', 8, data1, features, 'new_cases_per_million')

Time executing backward stepwise regression is 6.06s
                              OLS Regression Results                             
Dep. Variable:     new_cases_per_million   R-squared:                       0.394
Model:                               OLS   Adj. R-squared:                  0.342
Method:                    Least Squares   F-statistic:                     7.500
Date:                   Thu, 02 Jun 2022   Prob (F-statistic):           1.10e-12
Time:                           07:28:47   Log-Likelihood:                -1659.0
No. Observations:                    189   AIC:                             3350.
Df Residuals:                        173   BIC:                             3402.
Df Model:                             15                                         
Covariance Type:               nonrobust                                         
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:117: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [ ]:
# new deaths prediction with forward model
step_wise_regression(model, 'backward', 8, data2, features, 'new_deaths_per_million')

Time executing backward stepwise regression is 6.2s
                              OLS Regression Results                              
Dep. Variable:     new_deaths_per_million   R-squared:                       0.426
Model:                                OLS   Adj. R-squared:                  0.376
Method:                     Least Squares   F-statistic:                     8.567
Date:                    Thu, 02 Jun 2022   Prob (F-statistic):           1.61e-14
Time:                            07:28:53   Log-Likelihood:                -524.32
No. Observations:                     189   AIC:                             1081.
Df Residuals:                         173   BIC:                             1133.
Df Model:                              15                                         
Covariance Type:                nonrobust                                         
                                      coef    std err          t      P>|t|      [0.025      0.975]
------------------

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/tsatools.py:117: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


#6. Compare step-wise backward and forward regression

**1.   Method**
*   **Forward regression:** begins with a Null Model then starts adding the most significant variables one after the other until a pre-specified stopping rule is reached or all the variables under consideration are included in the model.
*   **Backward regression:** begins with a Full Model then starts removing the lease significant variables one after the other until a pre-specified stopping rule is reached or no variable is left in the model.

**2. Usage**
*   **Forward stepwise** is superior when the number of variables (features) under consideration is very large, only less than the sample size.
*   **Backward stepwise** is prefered in almost every other case as starting with the full model has the advantage of considering the effects of all variables simultaneously. This is especially important in case of collinearity because backward stepwise may be forced to keep them all in the model unlike forward selection where none of them might be entered.
* Hence unless the number of features is really large, use a **backward stepwise** approach.

**3. Results**
* In this project, given the same criteria (accuracy,etc.), backward stepwise and forward stepwise will return exactly the same 15 features (with different statistics) with the difference of time not considerable between two methods as the number of features are small. The aforementioned advantage of backward stepwise is not in use here as the relationship between variables are not significant. Also, forward stepwise is not better here as the number of attributes are relatively small compared to the number of records.